## Module 3 - In this jupyter notebook, perfomance indicators are calculated 
* Step 3a - Set up: Import modules/libraries, inport data, create output folder
* Step 3b - Calculate uniformity
* Step 3c - Calculate efficiency (beneficial fraction)
* Step 3d - Calculate adquacy
* Step 3e - Calculate relative water deficit 
**=====================================================================================================================**

![title](img/Fig3_1.png)

**=====================================================================================================================**
#### <span style='background :lightgreen' > References:
* Karimi, P., Bongani, B., Blatchford, M., and de Fraiture, C.: Global satellite-based ET products for the local level irrigation management: An application of irrigation performance assessment in the sugarbelt of Swaziland, Remote Sensing, 11, 705, 2019.
* Bastiaanssen, W. G., and Bos, M.: Irrigation performance indicators based on remotely sensed data: a review of literature, Irrigation and drainage systems, 13, 291-311, 1999.
* Bastiaanssen, W. G., Van der Wal, T., and Visser, T.: Diagnosis of regional evaporation by remote sensing to support irrigation performance assessment, Irrigation and Drainage Systems, 10, 1-23, 1996.

## Step 3a - Set up

## i) Import packages/libraries

In [2]:
import os
import sys
import glob

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

# change the directory to where the modules are saved
os.chdir(os.path.join(os.path.split(os.getcwd())[0], "Modules"))
from GIS_functions import GIS_function as gis

## ii) Import the input data
* Seasonal T, AETI, ETp 

In [3]:
# get seasonal data
#T_fhs    = glob.glob(r'..\Data\2L2_T_season\*.tif')
T_fhs    = glob.glob(r'C:\DATA\eLEAF\WaPOR\Gezira\Data Compoenets\T\*.tif')
#AETI_fhs = glob.glob(r'..\Data\2L2_AETI_season\*.tif')
AETI_fhs = glob.glob(r'C:\DATA\eLEAF\WaPOR\Gezira\Data Compoenets\AETI\*.tif')
ETp_fhs  = glob.glob(r'..\Data\2L1_ETp_season\*.tif')

## iii) Output folder: Make one or connect to the existing one

In [22]:
# output_folder
#output_folderBF = r'..\Data\Results\3_BeneficialFraction'  # create output folder
output_folderBF = r'..\Data\Results\3_BeneficialFraction' 
output_folderAd = r'..\Data\Results\3_Adequacy'  # create output folder
output_folderCWD = r'..\Data\Results\3_CWD'  # create output folder

# Make one if the folder does not exit
if not os.path.exists(output_folderBF):
    os.makedirs(output_folderBF)
if not os.path.exists(output_folderAd):
    os.makedirs(output_folderAd)
if not os.path.exists(output_folderCWD):
    os.makedirs(output_folderCWD)

## Step 3b - Calculate uniformity of water consumption
* Equity is defined as the coefficients of variation (CV) of seasonal ETa in the area of interest.
* It measures the evenness of the water supply in an irrigation scheme. 
* Note: CV of 0 to 10% is good, 10 to 25% is fair and CV > 25% is poor uniformity (Bastiaanssen et al., 1996) 
<br/>

In [8]:
# Uniformity of water Consumption 
# get seasonal AETI
# AETI_fhs = glob.glob(r'..\Data\2L2_AETI_season\*.tif') 

for i in range(len(AETI_fhs)):
    AETI = gis.OpenAsArray(AETI_fhs[i], nan_values=True)
    
    AETIm   = np.nanmean(AETI)
    AETIsd  = np.nanstd(AETI)
    
    CV_AETI = (AETIsd / AETIm) * 100
    
    # Identify the date from the file name
    date  = os.path.basename(AETI_fhs[i]).split('.')[0].replace('AETI', '').replace('_', ' ') 
    
    if CV_AETI < 10:
        U = 'Good Uniformity'
    if (CV_AETI >= 10) and (CV_AETI < 25):
        U = 'Fair Uniformity'
    else: 
        U = 'Poor Uniformity'

    print ('CV of AETI in', date, '=', round(CV_AETI, 1), ',', U)

CV of AETI in L3 GEZ  2101 = 64.3 , Poor Uniformity
CV of AETI in L3 GEZ  2102 = 70.2 , Poor Uniformity
CV of AETI in L3 GEZ  2103 = 70.2 , Poor Uniformity
CV of AETI in L3 GEZ  2104 = 71.7 , Poor Uniformity
CV of AETI in L3 GEZ  2105 = 73.9 , Poor Uniformity
CV of AETI in L3 GEZ  2106 = 74.7 , Poor Uniformity
CV of AETI in L3 GEZ  2107 = 72.8 , Poor Uniformity
CV of AETI in L3 GEZ  2108 = 70.5 , Poor Uniformity
CV of AETI in L3 GEZ  2109 = 73.1 , Poor Uniformity
CV of AETI in L3 GEZ  2110 = 76.3 , Poor Uniformity
CV of AETI in L3 GEZ  2111 = 77.9 , Poor Uniformity
CV of AETI in L3 GEZ  2112 = 73.7 , Poor Uniformity
CV of AETI in L3 GEZ  2113 = 77.9 , Poor Uniformity
CV of AETI in L3 GEZ  2114 = 76.5 , Poor Uniformity
CV of AETI in L3 GEZ  2115 = 74.2 , Poor Uniformity
CV of AETI in L3 GEZ  2116 = 71.1 , Poor Uniformity
CV of AETI in L3 GEZ  2117 = 74.6 , Poor Uniformity


## Step 3c - Calculate efficiency (beneficial fraction)
* Beneficial fraction is the ratio of the water that is consumed as transpiration compared to overall field water consumption (ETa). 
* $Beneficial fraction = \frac{T_a}{ET_a}$
* It is a measure of the efficiency of on farm water and agronomic practices in use of water for crop growth.

In [4]:
# collecting Geoinfo such as projection, the x and y axis
in_fh = AETI_fhs[0]      
driver, NDV, xsize, ysize, GeoT, Projection = gis.GetGeoInfo(in_fh)
spatial_extent = (GeoT[0], GeoT[0] + GeoT[1] * xsize, GeoT[3] + GeoT[5] * ysize, GeoT[3])  # get spatial extent of raster
  
for Tfh, ETfh in zip(T_fhs, AETI_fhs):
    T    = gis.OpenAsArray(Tfh,  nan_values=True)
    AETI = gis.OpenAsArray(ETfh, nan_values=True)
    
    T_over_AETI = T/AETI
    
    T_over_AETI_mean = np.nanmean(T_over_AETI)
    AETI_mean = np.nanmean(AETI)
    T_mean = np.nanmean(T)
    
    # Identify the date from the file name
    date  = os.path.basename(ETfh).split('.')[0].replace('AETI', '').replace('_', ' ') 
    
    #print ('Beneficial Fraction', date, '=', round(AETI_mean, 2))
    print ('BF, AETI, T respectivly of ', date, '=', 'Are', round(T_over_AETI_mean, 2), round(AETI_mean, 2), round(T_mean, 2))
    
    #update the file name, and save into output folder
    #basename  = os.path.basename(ETfh).replace('AETI', 'BF')  
    #output_fn = os.path.join(output_folderBF, basename)
    #gis.CreateGeoTiff(output_fn, T_over_AETI, driver, NDV, xsize, ysize, GeoT, Projection) 
    
    # Plot the raster map
    #seasonal = T_over_AETI
    
    #plt.figure(figsize = (12,8))
    #plt.imshow(seasonal, cmap='RdYlGn', vmin=np.nanmin(seasonal), vmax=np.nanmax(seasonal), extent=spatial_extent)
    #plt.colorbar(shrink=0.75, label='BF [-]')
    #plt.xlabel('Longitude ($^{\circ}$ East)', fontsize=14)  # add axes label
    #plt.ylabel('Latitude ($^{\circ}$ North)', fontsize=14)
    #plt.title('Beneficial fraction [-] ' + date)
    #plt.clim(0,1)
    #plt.show ()

BF, AETI, T respectivly of  L3 GEZ  1001 = Are 0.67 34.43 24.27
BF, AETI, T respectivly of  L3 GEZ  1002 = Are 0.73 39.98 30.0
BF, AETI, T respectivly of  L3 GEZ  1003 = Are 0.76 44.63 35.09


C:\Users\Safi\.conda\envs\waterpip\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.


BF, AETI, T respectivly of  L3 GEZ  1004 = Are 0.78 38.15 30.6
BF, AETI, T respectivly of  L3 GEZ  1005 = Are 0.71 39.38 29.79
BF, AETI, T respectivly of  L3 GEZ  1006 = Are 0.63 26.92 19.52
BF, AETI, T respectivly of  L3 GEZ  1007 = Are 0.59 30.5 20.9
BF, AETI, T respectivly of  L3 GEZ  1008 = Are 0.55 28.58 18.22
BF, AETI, T respectivly of  L3 GEZ  1009 = Are 0.61 29.07 19.09
BF, AETI, T respectivly of  L3 GEZ  1010 = Are 0.54 24.1 13.92
BF, AETI, T respectivly of  L3 GEZ  1011 = Are 0.5 16.88 9.02
BF, AETI, T respectivly of  L3 GEZ  1012 = Are 0.44 19.4 8.98
BF, AETI, T respectivly of  L3 GEZ  1013 = Are 0.39 20.4 8.45
BF, AETI, T respectivly of  L3 GEZ  1014 = Are 0.36 15.04 5.67
BF, AETI, T respectivly of  L3 GEZ  1015 = Are 0.28 18.9 5.66
BF, AETI, T respectivly of  L3 GEZ  1016 = Are 0.25 19.93 5.22
BF, AETI, T respectivly of  L3 GEZ  1017 = Are 0.18 23.38 4.36
BF, AETI, T respectivly of  L3 GEZ  1018 = Are 0.22 21.31 4.82
BF, AETI, T respectivly of  L3 GEZ  1019 = Are 0.35 21.1

BF, AETI, T respectivly of  L3 GEZ  1326 = Are 0.67 39.35 28.04
BF, AETI, T respectivly of  L3 GEZ  1327 = Are 0.75 35.64 28.54
BF, AETI, T respectivly of  L3 GEZ  1328 = Are 0.78 27.83 22.83
BF, AETI, T respectivly of  L3 GEZ  1329 = Are 0.74 30.6 24.48
BF, AETI, T respectivly of  L3 GEZ  1330 = Are 0.75 33.18 26.68
BF, AETI, T respectivly of  L3 GEZ  1331 = Are 0.69 36.13 28.04
BF, AETI, T respectivly of  L3 GEZ  1332 = Are 0.71 33.23 26.36
BF, AETI, T respectivly of  L3 GEZ  1333 = Are 0.73 28.22 22.25
BF, AETI, T respectivly of  L3 GEZ  1334 = Are 0.69 34.53 26.33
BF, AETI, T respectivly of  L3 GEZ  1335 = Are 0.79 25.85 21.44
BF, AETI, T respectivly of  L3 GEZ  1336 = Are 0.75 32.57 26.17
BF, AETI, T respectivly of  L3 GEZ  1401 = Are 0.74 32.17 25.83
BF, AETI, T respectivly of  L3 GEZ  1402 = Are 0.72 35.7 28.13
BF, AETI, T respectivly of  L3 GEZ  1403 = Are 0.69 47.81 35.98
BF, AETI, T respectivly of  L3 GEZ  1404 = Are 0.73 39.86 31.96
BF, AETI, T respectivly of  L3 GEZ  1405 =

BF, AETI, T respectivly of  L3 GEZ  1713 = Are 0.74 3.57 2.27
BF, AETI, T respectivly of  L3 GEZ  1714 = Are 0.46 7.81 3.41
BF, AETI, T respectivly of  L3 GEZ  1715 = Are 0.45 10.07 4.1
BF, AETI, T respectivly of  L3 GEZ  1716 = Are 0.38 6.65 2.64
BF, AETI, T respectivly of  L3 GEZ  1717 = Are 0.28 17.19 5.38
BF, AETI, T respectivly of  L3 GEZ  1718 = Are 0.26 14.53 4.3
BF, AETI, T respectivly of  L3 GEZ  1719 = Are 0.31 14.24 4.91
BF, AETI, T respectivly of  L3 GEZ  1720 = Are 0.24 28.59 7.59
BF, AETI, T respectivly of  L3 GEZ  1721 = Are 0.32 28.52 10.01
BF, AETI, T respectivly of  L3 GEZ  1722 = Are 0.39 24.24 10.14
BF, AETI, T respectivly of  L3 GEZ  1723 = Are 0.44 27.09 12.64
BF, AETI, T respectivly of  L3 GEZ  1724 = Are 0.53 26.82 15.16
BF, AETI, T respectivly of  L3 GEZ  1725 = Are 0.63 28.39 19.31
BF, AETI, T respectivly of  L3 GEZ  1726 = Are 0.66 39.78 27.81
BF, AETI, T respectivly of  L3 GEZ  1727 = Are 0.67 43.17 30.42
BF, AETI, T respectivly of  L3 GEZ  1728 = Are 0.63 4

BF, AETI, T respectivly of  L3 GEZ  2036 = Are 0.88 20.31 17.75
BF, AETI, T respectivly of  L3 GEZ  2101 = Are 0.88 19.56 17.27
BF, AETI, T respectivly of  L3 GEZ  2102 = Are 0.88 18.59 16.42
BF, AETI, T respectivly of  L3 GEZ  2103 = Are 0.88 21.7 19.18
BF, AETI, T respectivly of  L3 GEZ  2104 = Are 0.88 20.41 18.12
BF, AETI, T respectivly of  L3 GEZ  2105 = Are 0.86 20.18 17.66
BF, AETI, T respectivly of  L3 GEZ  2106 = Are 0.86 16.67 14.7
BF, AETI, T respectivly of  L3 GEZ  2107 = Are 0.83 21.52 18.47
BF, AETI, T respectivly of  L3 GEZ  2108 = Are 0.75 19.86 15.73
BF, AETI, T respectivly of  L3 GEZ  2109 = Are 0.73 17.86 13.48
BF, AETI, T respectivly of  L3 GEZ  2110 = Are 0.7 14.58 10.53
BF, AETI, T respectivly of  L3 GEZ  2111 = Are 0.63 13.35 8.95
BF, AETI, T respectivly of  L3 GEZ  2112 = Are 0.51 11.43 6.33
BF, AETI, T respectivly of  L3 GEZ  2113 = Are 0.59 8.6 5.01
BF, AETI, T respectivly of  L3 GEZ  2114 = Are 0.55 9.81 5.45
BF, AETI, T respectivly of  L3 GEZ  2115 = Are 0.4

## Step 3d - Calculate adequacy (relative evapotranspiration)
$Adequacy= \frac{ET_a}{ET_p}$

In [13]:
## Calculate and save raster adequacy layer

# collecting Geoinfo such as projection, the x and y axis
in_fh = AETI_fhs[0]      
driver, NDV, xsize, ysize, GeoT, Projection = gis.GetGeoInfo(in_fh)
spatial_extent = (GeoT[0], GeoT[0] + GeoT[1] * xsize, GeoT[3] + GeoT[5] * ysize, GeoT[3])  # get spatial extent of raster

for ETfh, ETpfh in zip(AETI_fhs, ETp_fhs):
    AETI = gis.OpenAsArray(ETfh,  nan_values=True)
    for i in range(len(AETI_fhs)):
        AETI = gis.OpenAsArray(AETI_fhs[i], nan_values=True)
        AETI1_1D  = np.reshape(AETI,  AETI.shape[0]*AETI.shape[1])
        ETp = np.nanpercentile(AETI1_1D, 90)
        #ETp  = gis.OpenAsArray(ETpfh, nan_values=True)
        
        ETa_by_ETp = (AETI / ETp)*100
        
        ETa_by_ETp_mean = np.nanmean(ETa_by_ETp)
        # Identify the date from the file name
        date  = os.path.basename(ETfh).split('.')[0].replace('AETI', '').replace('_', ' ')
        
        #print ('Adequacy is', date, '=', round(ETa_by_ETp, 2))
    print ('Adequacy is', date, '=', ETa_by_ETp_mean) 
        
    # update the file name, and save into output folder
    #basename  = os.path.basename(ETfh).replace('AETI', 'Adequacy')
    #output_fn = os.path.join(output_folderAd, basename)
    #gis.CreateGeoTiff(output_fn, ETa_by_ETp, driver, NDV, xsize, ysize, GeoT, Projection) 
  
    # Plot the raster map
    #seasonal = ETa_by_ETp
    
    #plt.figure(figsize = (12,8))
    #plt.imshow(seasonal, cmap='RdYlGn', vmin=np.nanmin(seasonal), vmax=np.nanmax(seasonal), extent=spatial_extent)
    #plt.colorbar(shrink=0.75, label='Adequacy [-]')
    #plt.xlabel('Longitude ($^{\circ}$ East)', fontsize=12)  # add axes label
    #plt.ylabel('Latitude ($^{\circ}$ North)', fontsize=12)
    #plt.title('Adequacy [-] ' + date)
    #plt.show ()
    ;

Adequacy is L3 GEZ  2101 = 58.473164
Adequacy is L3 GEZ  2102 = 58.473164
Adequacy is L3 GEZ  2103 = 58.473164
Adequacy is L3 GEZ  2104 = 58.473164
Adequacy is L3 GEZ  2105 = 58.473164
Adequacy is L3 GEZ  2106 = 58.473164


## Step 3e - Calculate Relative water Deficit (RWD)
$RWD= 1-\frac{ET_a}{ET_x}$
<br/>${ET_x} = $ Can be ETp or 99 percentile of the actual evapotranspiration

In [12]:
# collecting Geoinfo such as projection, the x and y axis
in_fh = AETI_fhs[0]      
driver, NDV, xsize, ysize, GeoT, Projection = gis.GetGeoInfo(in_fh)
spatial_extent = (GeoT[0], GeoT[0] + GeoT[1] * xsize, GeoT[3] + GeoT[5] * ysize, GeoT[3])  # get spatial extent of raster
   
for i in range(len(AETI_fhs)):
    AETI = gis.OpenAsArray(AETI_fhs[i], nan_values=True)
    
    # reshape the arrays
    AETI1_1D  = np.reshape(AETI,  AETI.shape[0]*AETI.shape[1])
    ETx       = np.nanpercentile(AETI1_1D, 95)
    
    AETI_mean = np.nanmean(AETI)
   
    RWD = 1-(AETI_mean/ETx)
    
    # Identify the date from the file name
    date  = os.path.basename(AETI_fhs[i]).split('.')[0].replace('AETI', '').replace('_', ' ') 
    
    #print ('Relative water deficit', date, '=', round(RWD, 2))
    print ('AETI mean', date, '=', round(AETI_mean, 2))

AETI mean L3 GEZ  2101 = 19.56
AETI mean L3 GEZ  2102 = 18.59
AETI mean L3 GEZ  2103 = 21.7
AETI mean L3 GEZ  2104 = 20.41
AETI mean L3 GEZ  2105 = 20.18
AETI mean L3 GEZ  2106 = 16.67
AETI mean L3 GEZ  2107 = 21.52
AETI mean L3 GEZ  2108 = 19.86
AETI mean L3 GEZ  2109 = 17.86
AETI mean L3 GEZ  2110 = 14.58
AETI mean L3 GEZ  2111 = 13.35
AETI mean L3 GEZ  2112 = 11.43
AETI mean L3 GEZ  2113 = 8.6
AETI mean L3 GEZ  2114 = 9.81
AETI mean L3 GEZ  2115 = 11.64
AETI mean L3 GEZ  2116 = 7.08
AETI mean L3 GEZ  2117 = 5.85


## Step 3f - Calculate Crop Water Dificit (CWD)
* CWD = ET-ETp
* ETp is the 95 percentile of the actual evapotranspiration

In [ ]:
# collecting Geoinfo such as projection, the x and y axis
in_fh = AETI_fhs[0]      
driver, NDV, xsize, ysize, GeoT, Projection = gis.GetGeoInfo(in_fh)
spatial_extent = (GeoT[0], GeoT[0] + GeoT[1] * xsize, GeoT[3] + GeoT[5] * ysize, GeoT[3])  # get spatial extent of raster
  
for Tfh, ETfh in zip(T_fhs, AETI_fhs):
    T    = gis.OpenAsArray(Tfh,  nan_values=True)
    AETI = gis.OpenAsArray(ETfh, nan_values=True)
    AETI1_1D  = np.reshape(AETI,  AETI.shape[0]*AETI.shape[1])
    ETp = np.nanpercentile(AETI1_1D, 90)
    
    CWD = AETI-ETp
    CWD_mean = np.nanmean(CWD)
    
    # Identify the date from the file name
    date  = os.path.basename(ETfh).split('.')[0].replace('AETI', '').replace('_', ' ') 
    
    # update the file name, and save into output folder
    basename  = os.path.basename(ETfh).replace('AETI', 'CWD')  
    output_fn = os.path.join(output_folderCWD, basename)
    gis.CreateGeoTiff(output_fn, T_over_AETI, driver, NDV, xsize, ysize, GeoT, Projection) 
    
    # Plot the raster map
    seasonal = CWD
    
    plt.figure(figsize = (12,8))
    plt.imshow(seasonal, cmap='RdYlGn', vmin=np.nanmin(seasonal), vmax=np.nanmax(seasonal), extent=spatial_extent)
    plt.colorbar(shrink=0.75, label='CWD [mm/dekade]')
    plt.xlabel('Longitude ($^{\circ}$ East)', fontsize=14)  # add axes label
    plt.ylabel('Latitude ($^{\circ}$ North)', fontsize=14)
    plt.title('Crop Water Deficit [-] ' + date)
#     plt.clim(0,1)
    plt.show ()
    print ('Relative water deficit', date, '=', CWD_mean)
;

## Step 3g - Calculate Reliablity

* Temporal variation of ET/ETp
* ETp is the 95 percentile of the actual evapotranspiration

## Step 3h - Calculate Land Productivity
* Y = AGBP/ET

## Step 3i - Calculate WP
* WP = Y/ET

## Step 3j - Calculate Crop Water Productivity per Beneficial Water Consumption (WPBF)
* WPBF = Y/T